# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv data into dataframe
data="../output_data/cities_data.csv"
data_df=pd.read_csv(data)
data_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Flin Flon,54.77,-101.86,38.30,80,100,4.52,CA,1601619716
1,Dolores,-36.31,-57.68,39.61,91,0,8.46,AR,1601619720
2,Orlik,52.52,99.83,30.83,73,100,4.99,RU,1601619721
3,Jesup,31.61,-81.89,60.80,87,1,4.97,US,1601619722
4,San Felipe,10.34,-68.74,71.56,91,95,2.21,VE,1601619626
...,...,...,...,...,...,...,...,...,...
655,Pacific Grove,36.62,-121.92,64.40,67,90,4.07,US,1601627628
656,Korem,12.51,39.52,67.86,46,2,10.16,ET,1601627647
657,Port Augusta,-32.50,137.77,89.60,15,0,21.92,AU,1601627574
658,Tingi,-11.30,35.03,83.82,39,5,6.29,TZ,1601627649


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Use Lat and Lng as location and humidity as the weight
locations=data_df[["Lat","Lng"]].astype(float)
humidity=data_df["Humidity"].astype(float)

                   

In [5]:
# Create a humdity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities to fit weather conditions
# Drop cities with a maximum temp above 100 F and less than 70 F
ideal_df=data_df.drop(data_df[data_df['Max_Temp'] > 100].index , inplace=False)
ideal_df=ideal_df.drop(ideal_df[ideal_df['Max_Temp'] < 70].index , inplace=False)
# Drop cities with wind speed greater than 20 mph
ideal_df=ideal_df.drop(ideal_df[ideal_df['Wind_Speed']>20].index, inplace=False)
# Drop citis with cloudiness greater than 70
ideal_df=ideal_df.drop(ideal_df[ideal_df['Cloudiness']>70].index, inplace=False)
ideal_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
7,Kushima,31.46,131.23,80.60,65,20,9.17,JP,1601619790
9,Dali,25.70,100.18,77.56,52,35,5.39,CN,1601619820
13,Terrasini,38.15,13.08,70.00,82,5,4.70,IT,1601619881
15,Kapaa,22.08,-159.32,78.80,78,1,4.70,US,1601620125
16,Arlit,18.74,7.39,90.64,33,58,7.96,NE,1601620150
...,...,...,...,...,...,...,...,...,...
651,Arroyos y Esteros,-25.07,-57.10,82.40,51,3,8.05,PY,1601627616
653,Conceição do Araguaia,-8.26,-49.26,75.88,63,8,3.80,BR,1601627624
654,Trairi,-3.28,-39.27,76.06,75,62,11.74,BR,1601627627
658,Tingi,-11.30,35.03,83.82,39,5,6.29,TZ,1601627649


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create `hotel_df`with "Hotel Name" column
hotel_df=ideal_df[['City', 'Country', 'Lat', 'Lng']].reset_index()
hotel_df['Hotel Name']=""
del hotel_df['index']
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Kushima,JP,31.46,131.23,
1,Dali,CN,25.70,100.18,
2,Terrasini,IT,38.15,13.08,
3,Kapaa,US,22.08,-159.32,
4,Arlit,NE,18.74,7.39,
...,...,...,...,...,...
196,Arroyos y Esteros,PY,-25.07,-57.10,
197,Conceição do Araguaia,BR,-8.26,-49.26,
198,Trairi,BR,-3.28,-39.27,
199,Tingi,TZ,-11.30,35.03,


In [8]:
# Define api search parameters
key=g_key
radius="5000"
type="lodging"
keyword="hotel"
rankby="distance"

# Loop through the coordinates to find the nearewst hotel in each city
for index, row in hotel_df.iterrows():
    
    # Define search url
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    # Define coordinates for loop
    lat = str(row['Lat'])
    lng = str(row['Lng'])
    
    # Update coordinates
    location = f"{lat},{lng}"
    
    # Query url
    query_url = url + "location="+ location + "&radius=" + radius + "&type=" + "lodging" + "&keyword=" + keyword + "&key="+key
    
    # Make request
    hotel_data= requests.get(query_url)
    
    # Convert to json
    hotel_data = hotel_data.json()
    try:
        print("retreiving data")
        hotel_df.loc[index, "Lat"] = hotel_data["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = hotel_data["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "Hotel Name"]= hotel_data["results"][0]["name"]
    except:
        "Data not found"

# Display dataframe with hotel data
hotel_df

retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreiving data
retreivi

,City,Country,Lat,Lng,Hotel Name
0,Kushima,JP,31.456156,131.176967,ささやかなおうち串間/高松(貸別荘)
1,Dali,CN,25.716920,100.135280,Honor Hotels & Resorts Yun Shu Dali
2,Terrasini,IT,38.166944,13.094722,Magaggiari Hotel Resort
3,Kapaa,US,22.042287,-159.337013,Hilton Garden Inn Kauai Wailua Bay
4,Arlit,NE,18.739003,7.389477,Hôtel Telwa Bungalow
...,...,...,...,...,...
196,Arroyos y Esteros,PY,-25.054852,-57.098122,Hotel Olivares
197,Conceição do Araguaia,BR,-8.272012,-49.260772,Tarumã Tropical Hotel
198,Trairi,BR,-3.239148,-39.232290,Rede Beach: Hotel em Guajiru
199,Tingi,TZ,-11.300000,35.030000,


In [14]:
# Drop rows with mmissing hotel names
hotel_df = hotel_df.dropna(axis = 0, how ='any')

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Kushima,JP,31.456156,131.176967,ささやかなおうち串間/高松(貸別荘)
1,Dali,CN,25.716920,100.135280,Honor Hotels & Resorts Yun Shu Dali
2,Terrasini,IT,38.166944,13.094722,Magaggiari Hotel Resort
3,Kapaa,US,22.042287,-159.337013,Hilton Garden Inn Kauai Wailua Bay
4,Arlit,NE,18.739003,7.389477,Hôtel Telwa Bungalow
...,...,...,...,...,...
196,Arroyos y Esteros,PY,-25.054852,-57.098122,Hotel Olivares
197,Conceição do Araguaia,BR,-8.272012,-49.260772,Tarumã Tropical Hotel
198,Trairi,BR,-3.239148,-39.232290,Rede Beach: Hotel em Guajiru
199,Tingi,TZ,-11.300000,35.030000,


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Kushima,JP,31.456156,131.176967,ささやかなおうち串間/高松(貸別荘)
1,Dali,CN,25.716920,100.135280,Honor Hotels & Resorts Yun Shu Dali
2,Terrasini,IT,38.166944,13.094722,Magaggiari Hotel Resort
3,Kapaa,US,22.042287,-159.337013,Hilton Garden Inn Kauai Wailua Bay
4,Arlit,NE,18.739003,7.389477,Hôtel Telwa Bungalow
...,...,...,...,...,...
196,Arroyos y Esteros,PY,-25.054852,-57.098122,Hotel Olivares
197,Conceição do Araguaia,BR,-8.272012,-49.260772,Tarumã Tropical Hotel
198,Trairi,BR,-3.239148,-39.232290,Rede Beach: Hotel em Guajiru
199,Tingi,TZ,-11.300000,35.030000,
